# A minimal example to demonstrate how the trainer for FNet and wGAN GP plus the callbacks works along with patched dataset

Is will not be dependent on the pe2loaddata generated index file from the ALSF pilot data repo unlike the other example notebook

In [1]:
import sys
import pathlib

import pandas as pd
import torch.nn as nn
import torch.optim as optim

sys.path.append(str(pathlib.Path('.').absolute().parent.parent))
print(str(pathlib.Path('.').absolute().parent.parent))

## Dataset
from virtual_stain_flow.datasets.GenericImageDataset import GenericImageDataset
from virtual_stain_flow.datasets.CachedDataset import CachedDataset

## FNet training
from virtual_stain_flow.models.fnet import FNet
from virtual_stain_flow.trainers.Trainer import Trainer

## wGaN training
from virtual_stain_flow.models.unet import UNet
from virtual_stain_flow.models.discriminator import GlobalDiscriminator
from virtual_stain_flow.trainers.WGANTrainer import WGANTrainer

## wGaN losses
from virtual_stain_flow.losses.GradientPenaltyLoss import GradientPenaltyLoss
from virtual_stain_flow.losses.DiscriminatorLoss import WassersteinLoss
from virtual_stain_flow.losses.GeneratorLoss import GeneratorLoss

from virtual_stain_flow.transforms.MinMaxNormalize import MinMaxNormalize

## Metrics
from virtual_stain_flow.metrics.MetricsWrapper import MetricsWrapper
from virtual_stain_flow.metrics.PSNR import PSNR
from virtual_stain_flow.metrics.SSIM import SSIM

## callback
from virtual_stain_flow.callbacks.MlflowLogger import MlflowLogger
from virtual_stain_flow.callbacks.IntermediatePlot import IntermediatePlot


/home/weishanli/Waylab


/home/weishanli/anaconda3/envs/cp_gan_env/lib/python3.9/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## Specify train data and output paths

In [ ]:
EXAMPLE_PATCH_DATA_EXPORT_PATH = '/REPLACE/WITH/PATH/TO/DATA'

EXAMPLE_DIR = pathlib.Path('.').absolute() / 'example_train_generic_dataset'
EXAMPLE_DIR.mkdir(exist_ok=True)

In [3]:
!rm -rf example_train_generic_dataset/*

In [4]:
PLOT_DIR = EXAMPLE_DIR / 'plot'
PLOT_DIR.mkdir(parents=True, exist_ok=True)

MLFLOW_DIR =EXAMPLE_DIR / 'mlflow'
MLFLOW_DIR.mkdir(parents=True, exist_ok=True)

## Configure channels

In [5]:
channel_names = [
    "OrigBrightfield",
    "OrigDNA",
    "OrigER",
    "OrigMito",
    "OrigRNA",
    "OrigAGP",
]
input_channel_name = "OrigBrightfield"
target_channel_names = [ch for ch in channel_names if ch != input_channel_name]

## Prep Patch dataset and Cache

In [6]:
ds = GenericImageDataset(
    image_dir=EXAMPLE_PATCH_DATA_EXPORT_PATH,
    site_pattern=r"^([^_]+_[^_]+_[^_]+)",
    channel_pattern=r"_([^_]+)\.tiff$",
    verbose=True
)

## Set input and target channels
ds.set_input_channel_keys([input_channel_name])
ds.set_target_channel_keys('OrigDNA')

## Cache for faster training 
cds = CachedDataset(
    ds,
    prefill_cache=True
)

2025-03-03 20:30:03,223 - DEBUG - Channel keys: {'OrigMito', 'OrigBrightfield', 'OrigDNA', 'OrigRNA', 'OrigER', 'OrigAGP'} detected
2025-03-03 20:30:03,225 - DEBUG - No channel keys specified, skip
2025-03-03 20:30:03,226 - DEBUG - No channel keys specified, skip
2025-03-03 20:30:03,226 - DEBUG - Setting input transform ...
2025-03-03 20:30:03,226 - DEBUG - Setting target transform ...
2025-03-03 20:30:03,226 - DEBUG - Set input channel(s) as ['OrigBrightfield']
2025-03-03 20:30:03,227 - DEBUG - Set target channel(s) as ['OrigDNA']


# FNet trainer

## Train model without callback and check logs

In [7]:
model = FNet(depth=4)
lr = 3e-4
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))

trainer = Trainer(
    model = model,
    optimizer = optimizer,
    backprop_loss = nn.L1Loss(),
    dataset = cds,
    batch_size = 16,
    epochs = 10,
    patience = 5,
    callbacks=None,
    metrics={'psnr': PSNR(_metric_name="psnr"), 'ssim': SSIM(_metric_name="ssim")},
    device = 'cuda',
    early_termination_metric = None
)

trainer.train()

In [8]:
pd.DataFrame(trainer.log)

,epoch,L1Loss,val_L1Loss,psnr,ssim,val_psnr,val_ssim
0,1,1556.951280,1610.125305,-70.284943,9.063277e-10,-70.565842,-8.826771e-10
1,2,1702.884250,1610.080627,-70.887955,-3.853542e-09,-70.565781,-1.012693e-09
2,3,1515.534356,1609.984192,-70.115860,-4.895770e-09,-70.565659,-1.703562e-09
3,4,1559.352539,1609.861084,-70.398109,-3.283771e-09,-70.565506,-1.826288e-09
4,5,1545.541517,1609.824585,-70.343849,-3.253234e-09,-70.565460,-1.882144e-09
5,6,1542.607639,1609.786377,-70.497559,-8.243816e-10,-70.565414,-1.561940e-09
6,7,1501.347873,1609.757874,-69.796379,3.219659e-10,-70.565384,-1.503889e-09
7,8,1526.650323,1609.744629,-70.212173,-1.515618e-10,-70.565361,-9.222221e-10
8,9,1527.799533,1609.728699,-70.201828,-8.367405e-11,-70.565346,-1.539568e-09
9,10,1627.603136,1609.718262,-70.634064,-1.587237e-11,-70.565331,-7.939729e-10


## Train model with alternative early termination metric

In [9]:
model = FNet(depth=4)
lr = 3e-4
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))

trainer = Trainer(
    model = model,
    optimizer = optimizer,
    backprop_loss = nn.L1Loss(),
    dataset = cds,
    batch_size = 16,
    epochs = 10,
    patience = 5,
    callbacks=None,
    metrics={'psnr': PSNR(_metric_name="psnr"), 'ssim': SSIM(_metric_name="ssim")},
    device = 'cuda',
    early_termination_metric = 'psnr' # set early termination metric as psnr for the sake of demonstration
)

trainer.train()

Early termination at epoch 6 with best validation metric -69.82839965820312


## Train with mlflow logger callbacks

In [10]:
mlflow_logger_callback = MlflowLogger(
        name='mlflow_logger',
        mlflow_uri=MLFLOW_DIR / 'mlruns',
        mlflow_experiment_name='Default',
        mlflow_start_run_args={'run_name': 'example_train', 'nested': True},
        mlflow_log_params_args={
            'lr': 3e-4
        },
    )

del trainer

trainer = Trainer(
    model = model,
    optimizer = optimizer,
    backprop_loss = nn.L1Loss(),
    dataset = cds,
    batch_size = 16,
    epochs = 10,
    patience = 5,
    callbacks=[mlflow_logger_callback],
    metrics={'psnr': PSNR(_metric_name="psnr"), 'ssim': SSIM(_metric_name="ssim")},
    device = 'cuda'
)

trainer.train()

# wGaN GP example with mlflow logger callback and plot callback

In [11]:
generator = UNet(
    n_channels=1,
    n_classes=1
)

discriminator = GlobalDiscriminator(
    n_in_channels = 2,
    n_in_filters = 64,
    _conv_depth = 4,
    _pool_before_fc = True
)

generator_optimizer = optim.Adam(generator.parameters(), 
                                 lr=0.0002, 
                                 betas=(0., 0.9))
discriminator_optimizer = optim.Adam(discriminator.parameters(), 
                                     lr=0.00002, 
                                     betas=(0., 0.9),
                                     weight_decay=0.001)

gp_loss = GradientPenaltyLoss(
    _metric_name='gp_loss',
    discriminator=discriminator,
    weight=10.0,
)

gen_loss = GeneratorLoss(
    _metric_name='gen_loss'
)

disc_loss = WassersteinLoss(
    _metric_name='disc_loss'
)

mlflow_logger_callback = MlflowLogger(
        name='mlflow_logger',
        mlflow_uri=MLFLOW_DIR / 'mlruns',
        mlflow_experiment_name='Default',
        mlflow_start_run_args={'run_name': 'example_train_wgan', 'nested': True},
        mlflow_log_params_args={
            'gen_lr': 0.0002,
            'disc_lr': 0.00002
        },
    )

plot_callback = IntermediatePlot(
    name='plotter',
    path=PLOT_DIR,
    dataset=ds, # give it the patch dataset as opposed to the cached dataset
    indices=[1,3,5,7,9], # plot 5 selected patches images from the dataset
    plot_metrics=[SSIM(_metric_name='ssim'), PSNR(_metric_name='psnr')],
    figsize=(20, 25),
    show_plot=False,
)

wgan_trainer = WGANTrainer(
    dataset=cds,
    batch_size=16,
    epochs=20,
    patience=20, # setting this to prevent unwanted early termination here
    device='cuda',
    generator=generator,
    discriminator=discriminator,
    gen_optimizer=generator_optimizer,
    disc_optimizer=discriminator_optimizer,
    generator_loss_fn=gen_loss,
    discriminator_loss_fn=disc_loss,
    gradient_penalty_fn=gp_loss,
    discriminator_update_freq=1,
    generator_update_freq=2,
    callbacks=[mlflow_logger_callback, plot_callback],
    metrics={'ssim': SSIM(_metric_name='ssim'), 
             'psnr': PSNR(_metric_name='psnr')}
)

wgan_trainer.train()

del generator
del wgan_trainer

## # wGaN GP example with mlflow logger callback and alternative early termination loss

In [12]:
generator = UNet(
    n_channels=1,
    n_classes=1
)

discriminator = GlobalDiscriminator(
    n_in_channels = 2,
    n_in_filters = 64,
    _conv_depth = 4,
    _pool_before_fc = True
)

generator_optimizer = optim.Adam(generator.parameters(), 
                                 lr=0.0002, 
                                 betas=(0., 0.9))
discriminator_optimizer = optim.Adam(discriminator.parameters(), 
                                     lr=0.00002, 
                                     betas=(0., 0.9),
                                     weight_decay=0.001)

gp_loss = GradientPenaltyLoss(
    _metric_name='gp_loss',
    discriminator=discriminator,
    weight=10.0,
)

gen_loss = GeneratorLoss(
    _metric_name='gen_loss'
)

disc_loss = WassersteinLoss(
    _metric_name='disc_loss'
)

mlflow_logger_callback = MlflowLogger(
        name='mlflow_logger',
        mlflow_uri=MLFLOW_DIR / 'mlruns',
        mlflow_experiment_name='Default',
        mlflow_start_run_args={'run_name': 'example_train_wgan_mae_early_term', 'nested': True},
        mlflow_log_params_args={
            'gen_lr': 0.0002,
            'disc_lr': 0.00002
        },
    )

wgan_trainer = WGANTrainer(
    dataset=cds,
    batch_size=16,
    epochs=20,
    patience=5, # lower patience here
    device='cuda',
    generator=generator,
    discriminator=discriminator,
    gen_optimizer=generator_optimizer,
    disc_optimizer=discriminator_optimizer,
    generator_loss_fn=gen_loss,
    discriminator_loss_fn=disc_loss,
    gradient_penalty_fn=gp_loss,
    discriminator_update_freq=1,
    generator_update_freq=2,
    callbacks=[mlflow_logger_callback],
    metrics={'ssim': SSIM(_metric_name='ssim'), 
             'psnr': PSNR(_metric_name='psnr'),
             'mae': MetricsWrapper(_metric_name='mae', module=nn.L1Loss()) # use a wrapper for torch nn L1Loss
             },
    early_termination_metric = 'mae' # update early temrination loss with the supplied L1Loss/mae metric instead of the default GaN generator loss
)

wgan_trainer.train()